***GENERATED CODE FOR gettingfile PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Disaster', 'transformation_label': 'String Indexer'}], 'feature': 'Disaster', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '190', 'mean': '', 'stddev': '', 'min': 'Arizona Severe Weather (October 2010)', 'max': 'Western/Southeast Wildfires (Summer-Fall 2016)', 'missing': '0', 'distinct': '188'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Disaster'}, {'feature_label': 'Disaster', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Disaster')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Disaster Type', 'transformation_label': 'String Indexer'}], 'feature': 'Disaster Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '190', 'mean': '', 'stddev': '', 'min': 'Drought', 'max': 'Winter Storm', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Disaster Type'}, {'feature_label': 'Disaster Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Disaster Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Beginning Date', 'transformation_label': 'String Indexer'}], 'feature': 'Beginning Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '190', 'mean': '', 'stddev': '', 'min': '1/1/1995', 'max': '9/6/2017', 'missing': '0', 'distinct': '187'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Beginning Date'}, {'feature_label': 'Beginning Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Beginning Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Ending Date', 'transformation_label': 'String Indexer'}], 'feature': 'Ending Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '190', 'mean': '', 'stddev': '', 'min': '1/11/1997', 'max': '9/9/2004', 'missing': '0', 'distinct': '185'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Ending Date'}, {'feature_label': 'Ending Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Ending Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Description', 'transformation_label': 'String Indexer'}], 'feature': 'Description', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '190', 'mean': '', 'stddev': '', 'min': '"Drought and heat wave conditions created major impacts across Texas, Oklahoma, New Mexico, Arizona, southern Kansas, and western Louisiana. In Texas and Oklahoma, a majority of range and pastures were classified in ""very poor"" condition for much of the 2011 crop growing season."',
                                                                                                                                                                                                                                                                        'max': 'Winter storm caused widespread damage across numerous Midwest, Southeast and Northeastern states (AL, GA, IL, IN, KY, MD, MI, MO, MS, NC, NJ, NY, OH, PA, SC, TN, VA).', 'missing': '0', 'distinct': '190'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Description'}, {'feature_label': 'Description', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Description')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Event ID', 'transformation_label': 'String Indexer'}], 'feature': 'Event ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '190', 'mean': '10095.99', 'stddev': '54.72', 'min': ' as prodigious amounts of rainfall were common in many locations. Florence made landfall as a category 1', 'max': '10190', 'missing': '0', 'distinct': '190'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Event ID'}, {'feature_label': 'Event ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Event ID')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run gettingfileHooks.ipynb
try:
	#sourcePreExecutionHook()

	disaster = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Disaster.csv', 'filename': 'Disaster.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run gettingfileHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(disaster,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "index", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "190", "mean": "94.5", "stddev": "54.99", "min": "0", "max": "189", "missing": "0"}, "updatedLabel": "index"}, {"transformationsData": [{"feature_label": "Disaster", "transformation_label": "String Indexer"}], "feature": "Disaster", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "190", "mean": "", "stddev": "", "min": "Arizona Severe Weather (October 2010)", "max": "Western/Southeast Wildfires (Summer-Fall 2016)", "missing": "0", "distinct": "188"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Disaster"}, {"transformationsData": [{"feature_label": "Disaster Type", "transformation_label": "String Indexer"}], "feature": "Disaster Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "190", "mean": "", "stddev": "", "min": "Drought", "max": "Winter Storm", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Disaster Type"}, {"transformationsData": [{"feature_label": "Beginning Date", "transformation_label": "String Indexer"}], "feature": "Beginning Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "190", "mean": "", "stddev": "", "min": "1/1/1995", "max": "9/6/2017", "missing": "0", "distinct": "187"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Beginning Date"}, {"transformationsData": [{"feature_label": "Ending Date", "transformation_label": "String Indexer"}], "feature": "Ending Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "190", "mean": "", "stddev": "", "min": "1/11/1997", "max": "9/9/2004", "missing": "0", "distinct": "185"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Ending Date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total CPI-Adjusted Cost (Millions of Dollars)", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "190", "mean": "6838.76", "stddev": "17819.57", "min": "1000.0", "max": "165012.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Total CPI-Adjusted Cost (..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Deaths", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "190", "mean": "50.8", "stddev": "256.46", "min": "0", "max": "2981", "missing": "0"}, "updatedLabel": "Deaths"}, {"transformationsData": [{"feature_label": "Description", "transformation_label": "String Indexer"}], "feature": "Description", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "190", "mean": "", "stddev": "", "min": "\"Drought and heat wave conditions created major impacts across Texas, Oklahoma, New Mexico, Arizona, southern Kansas, and western Louisiana. In Texas and Oklahoma, a majority of range and pastures were classified in \"\"very poor\"\" condition for much of the 2011 crop growing season.\"", "max": "Winter storm caused widespread damage across numerous Midwest, Southeast and Northeastern states (AL, GA, IL, IN, KY, MD, MI, MO, MS, NC, NJ, NY, OH, PA, SC, TN, VA).", "missing": "0", "distinct": "190"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Description"}, {"transformationsData": [{"feature_label": "Event ID", "transformation_label": "String Indexer"}], "feature": "Event ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "190", "mean": "10095.99", "stddev": "54.72", "min": " as prodigious amounts of rainfall were common in many locations. Florence made landfall as a category 1", "max": "10190", "missing": "0", "distinct": "190"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Event ID"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run gettingfileHooks.ipynb
try:
	#sinkPreExecutionHook()

	weatherdisastercostsanddeathoutput = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/65dcd21cd0444ca8ff5a3cdf/part-00000-a8bb7e53-4fc0-4907-81b1-fbefdd30410b-c000.csv', 'samplecount': 190, 'originalcount': 238, 'orignalKey': None, 'pathOnly': '/weather Disaster costs and Death', 'project_id': '65da2b413ee48657f7317309', 'parent_id': '65da2b413ee48657f7317309', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'index', 'alias': 'index', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Total CPI-Adjusted Cost (Millions of Dollars)', 'alias': 'Total CPI-Adjusted Cost (Millions of Dollars)', 'type': 'real', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Deaths', 'alias': 'Deaths', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Disaster_stringindexer', 'alias': 'Disaster_stringindexer', 'type': 'real', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Disaster Type_stringindexer', 'alias': 'Disaster Type_stringindexer', 'type': 'real', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Beginning Date_stringindexer', 'alias': 'Beginning Date_stringindexer', 'type': 'real', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Ending Date_stringindexer', 'alias': 'Ending Date_stringindexer', 'type': 'real', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Description_stringindexer', 'alias': 'Description_stringindexer', 'type': 'real', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Event ID_stringindexer', 'alias': 'Event ID_stringindexer', 'type': 'real', 'position': '8'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'index', 'alias': 'index', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Disaster', 'alias': 'Disaster', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Disaster Type', 'alias': 'Disaster Type', 'type': 'String', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Beginning Date', 'alias': 'Beginning Date', 'type': 'date', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Ending Date', 'alias': 'Ending Date', 'type': 'date', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Total CPI-Adjusted Cost (Millions of Dollars)', 'alias': 'Total CPI-Adjusted Cost (Millions of Dollars)', 'type': 'real', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Deaths', 'alias': 'Deaths', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Description', 'alias': 'Description', 'type': 'String', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', 'field': 'Event ID', 'alias': 'Event ID', 'type': 'String', 'position': '8'}], 'server': 'https://ml.colaberry.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'FileWeather.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-dd4c256c-fc01-4929-9f7a-2ce9d708bcc4-c000.csv', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
